# BOT AUTOCOMPARA

In [1]:
# Importamos las librerias

import time
from datetime import date, datetime
import pandas as pd
import os
#import fitz
#import glob
import re
import numpy as np
import warnings
#warnings.filterwarnings('ignore')



import locale
locale.setlocale(locale.LC_MONETARY, 'en_US.UFT-8')

import os
#from tika import parser
import re

#import threading
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
from Scrapeo_HDI import *

## Funciones

In [3]:
def ingresar_texto(find_scraper, id_elemento, xpath_elemento, class_name, texto):
    method_used = None
    try:
        try:
            # Intenta encontrar el elemento por ID
            element = find_scraper.id_s(id_elemento)
            method_used = 'ID'
        except:
            try:
                # Si no encuentra por ID, intenta por XPath
                element = find_scraper.xpath_s(xpath_elemento)
                method_used = 'XPath'
            except:
                # Finalmente, intenta por nombre de clase
                element = find_scraper.class_s(class_name)
                method_used = 'Class Name'

        element.clear()  # Limpia el campo de texto si es necesario
        element.send_keys(texto)  # Ingresa el texto deseado
        print(f"Elemento encontrado por: {method_used}")
    except Exception as e:
        print(f"Error al intentar escribir en el elemento: {e}. Método intentado: {method_used if method_used else 'Todos los métodos fallaron'}")



## Variables

# Scrapeo

In [4]:
CO = webdriver.ChromeOptions()

CO.add_experimental_option('excludeSwitches', ['enable-logging'])
CO.add_argument('--ignore-certificate-errors')
CO.add_argument('--start-maximized')
CO.add_argument("--disable-blink-features=AutomationControlled")
CO.add_experimental_option("excludeSwitches", ['enable-automation'])
CO.add_experimental_option('useAutomationExtension', False)
CO.add_experimental_option('useAutomationExtension', False)
CO.add_argument('--profile-directory=Default')
CO.add_argument("--disable-extensions")
CO.add_argument("--start-maximized")
CO.add_argument('--allow-insecure-localhost')
CO.add_argument('--allow-running-insecure-content')
CO.add_argument('--disable-gpu')
CO.add_argument('--disable-dev-shm-usage')
CO.add_argument('--ignore-certificate-errors')



## Diccionario Tipos (Auto/Pickup)

In [5]:
dict_tipo_vehiculo = {'AUTO':'auto/suv','PICK UP':'pickup'}
dict_sexo = {'Masculino':'1','Femenino':'2','Femenio':'2','M':'1','F':'2'}

dict_aseguradora = {
    'a_9': 'MAPFRE',
    'a_10': 'QUALITAS',
    'a_3': 'ANA',
    'a_1': 'ABA/CHUBB',
    'a_2': 'AIG',
    'a_6': 'GNP',
    'a_8': 'INBURSA',
    'a_11': 'ZURICH',
    'a_4': 'ATLAS',
    'a_5': 'AXA',
    'a_7': 'HDI'
}




In [6]:
tipo = 'AUTO'
año = '2024'
vehiculo = 'TIGGO 2 PRO CVT 3CIL 1.0L TURBO LUXURY'
sexo = 'Masculino'
'date','name','cp','email','phone'
info_cliente = {'date':'08/08/2000', 'name':'Juan', 'email':'juan.perez@gmail.com', 'phone':'5512345678','cp':'01000'}


tipo_vehiculo = dict_tipo_vehiculo[tipo]
num_sexo = dict_sexo[sexo]

In [7]:
def scrapeo_autocompara(tipo, año, vehiculo, sexo, info_cliente):
    driver = webdriver.Chrome(options = CO)
    driver.get("https://www.autocompara.com/")
    find_scraper = Find_Scrap(driver)
    find_wait_scraper =Find_Wait_Scrap(driver)

    def Mover_ScrollV(p):  
        driver.execute_script(f"window.scrollTo(0,{p})")


    tipo_vehiculo = dict_tipo_vehiculo[tipo]
    num_sexo = dict_sexo[sexo]
    find_scraper.xpath_s(f'//*[@id="{tipo_vehiculo}"]').click()

    if tipo in ['AUTO','PICK UP']:
        ## PARTE1
        # Ingresar Año del Vehiculo
        caja_año = find_scraper.xpath_s('//*[@id="year"]')
        caja_año.send_keys(año)
        
        # Ingresar Vehiculo
        caja_vehiculo = find_wait_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/div[3]/div[3]/ng-select/div/div/div[2]/input')
        caja_vehiculo.send_keys(vehiculo)
        caja_vehiculo.send_keys(Keys.ENTER)

        # Click en el boton de cotizar
        Mover_ScrollV(200)
        boton_cotizar = find_wait_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/div[3]/div[6]/div[2]/button')
        boton_cotizar.click()

        ## PARTE2
        # Boton Sexo
        Mover_ScrollV(0)
        boton_sexo = find_scraper.xpath_s(f'/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/ac-quotation-data/section/form/div/div[2]/div[2]/label[{num_sexo}]')
        boton_sexo.click()

        # Formulario cliente
        for info in ['date','name','cp','email','phone']:
            time.sleep(2)
            select_info = find_scraper.xpath_s(f'//*[@id="{info}"]') # Es la misma lógica para todos los datos del cliente
            select_info.send_keys(info_cliente[info])

        # Mover Scroll para poder interactuar con los elementos
        Mover_ScrollV(200)
        time.sleep(2)

        # Boton Aviso de Privacidad
        boton_aviso_privacidad = find_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/ac-quotation-data/section/div/div[2]/div/div/label')
        #boton_aviso_privacidad.click()
        try:
            error_aviso_privacidad = find_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/ac-quotation-data/section/div/div[2]/div/p')
            error_aviso_privacidad.text
            boton_aviso_privacidad.click()
        except:
            pass
            
        boton_cotizar = find_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-home/section/div[2]/div[2]/ac-vehicle-data/section/div/ac-quotation-data/section/div/div[3]/button')
        boton_cotizar.click()

        ## PARTE3

        # Mover el deducible DM al 5%
        try:
            time.sleep(5)
            deducible_dm = find_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-quotation/section/div[4]/div[1]/div[1]/div[3]/ng5-slider/span[12]/span[2]/ng5-slider-tooltip-wrapper[1]/div')
            actions = ActionChains(driver)
            actions.move_to_element(deducible_dm).click().perform()

            # Mover el deducible RT al 10%
            deducible_rt = find_scraper.xpath_s('/html/body/app-root/block-ui/div/div/ac-quotation/section/div[4]/div[1]/div[1]/div[4]/ng5-slider/span[12]/span[2]/ng5-slider-tooltip-wrapper[1]/div')
            actions = ActionChains(driver)
            actions.move_to_element(deducible_rt).click().perform()
        except:
            pass

        time.sleep(5)
        clase_dicts = {'AUTO':'cards','PICK UP':'quotes'}
        class_name = clase_dicts[tipo]

        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser")
        try:
            opciones_seguros = soup.find_all("div", {"class" : f"{class_name}"})
            op = opciones_seguros[0]
            text_aux = op.text
            text_aux.split('$')[1].split(' ')[0].split('\n')[0].replace(',','') 
        except:
            time.sleep(3)
            html_source = driver.page_source
            soup = BeautifulSoup(html_source,"html.parser")
            opciones_seguros = soup.find_all("div", {"class" : f"{class_name}"})

        mas_barato = 0
        mejor_servicio = 0
        precios = []
        aseguradoras = []
        if tipo =='AUTO':
            i = 1
            for opcion in opciones_seguros:
                texto = opcion.text
                precio = texto.split('$')[1].split(' ')[0].split('\n')[0].replace(',','') 
                aseguradora = str(opcion)
                aseguradora = aseguradora.split(' logo-insurance')[0][-4:].replace('"','')
                if 'barato' in texto:
                    mas_barato = aseguradora
                if 'servicio' in texto:
                    mejor_servicio = aseguradora
                precios.append(precio)
                aseguradoras.append(aseguradora)
                i+=1
                #print(f'Aseguradora: {aseguradora} Precio: {precio}')
        elif tipo == 'PICK UP':
            i = 0
            scrap_precios = find_scraper.class_p('quotes')
            for opcion in opciones_seguros:
                texto = scrap_precios[i].text
                precio = texto.split('$')[-1].split(' ')[0].split('\n')[1].replace(',','') 
                aseguradora = str(opcion)
                aseguradora = aseguradora.split(' logo-insurance')[0][-4:].replace('"','')
                if 'barato' in texto:
                    mas_barato = aseguradora
                if 'servicio' in texto:
                    mejor_servicio = aseguradora
                precios.append(precio)
                aseguradoras.append(aseguradora)
                i+=1
                #print(f'Aseguradora: {aseguradora} Precio: {precio}')


        aux = pd.DataFrame({'Aseguradora':aseguradoras,'Precio':precios})
        for k in info_cliente.keys():
            aux[k] = info_cliente[k]   
        aux.rename(columns={'date':'Fecha_Nacimiento','cp':'CP'},inplace=True)
        aux['Mas_Barato'] = mas_barato
        aux['Mejor_Servicio'] = mejor_servicio
        aux['DESCRIPCION'] = vehiculo
        aux['TIPO'] = tipo
        aux['MODELO'] = año
        aux['SEXO'] = sexo
        aux['BIRTHDATE'] = info_cliente['date']
        aux = aux[['DESCRIPCION','MODELO','CP','SEXO','BIRTHDATE','TIPO', 'Mas_Barato', 'Mejor_Servicio','Aseguradora','Precio']]
        for col in [ 'Mas_Barato', 'Mejor_Servicio','Aseguradora']:
            aux[col] = aux[col].map(dict_aseguradora)
        aux['Precio'] = aux['Precio'].astype(float)
        aux = aux.pivot_table(index=['DESCRIPCION','MODELO','TIPO','CP','SEXO','BIRTHDATE','Mas_Barato', 'Mejor_Servicio'], columns='Aseguradora', values='Precio' ).reset_index()
        driver.quit()
        return aux
    

In [8]:
import os
ruta = os.getcwd()
carpeta = 'Cotizar'
ruta_carpeta = os.path.join(ruta,carpeta)

acotizar = pd.read_csv(os.path.join(ruta_carpeta,'Muestra_Julio.csv'))
acotizar = pd.read_excel(os.path.join(ruta_carpeta,'Muestra.xlsx'),skiprows=1)

In [9]:
acotizar = acotizar.head(10)

In [10]:
cotizaciones = pd.DataFrame()
def Esperar_Cotizacion(minutos):
    for s in range(minutos,0,-1):
        print(f'...Intentando de nuevo en {s} min...')
        time.sleep(60)
    print(f'...Intentando de nuevo...')

for i in range(len(acotizar)):
    tipo = acotizar.loc[i,'TIPO']
    año = str(acotizar.loc[i,'MODELO'])
    vehiculo = acotizar.loc[i,'DESCRIPCION']
    sexo = acotizar.loc[i,'SEXO']
    anio_birthdate = acotizar.loc[i,'Anio']
    try:
        mes_birthdate = acotizar.loc[i,'Mes']
        dia_birthdate = acotizar.loc[i,'Dia']
    except:
        mes_birthdate = str(datetime.now().month)
        dia_birthdate = str(datetime.now().day)
        if len(mes_birthdate)<2:
            mes_birthdate = '0'+mes_birthdate
        if len(dia_birthdate)<2:
            dia_birthdate = '0'+dia_birthdate

    date = f'{dia_birthdate}/{mes_birthdate}/{anio_birthdate}'
    try:
        nombre = acotizar.loc[i,'Nombre']
    except:
        nombre = 'Omar'
    try:
        email = acotizar.loc[i,'Correo']
    except:
        email = 'omar.espinosa@gmail.com'
    telefono = '5512345678'
    cp = str(acotizar.loc[i,'CP'])
    if len(cp) < 5:
        cp = '0'*(5-len(cp)) + cp
    info_cliente = {'date':date, 'name':nombre, 'email':email, 'phone':telefono,'cp':cp}

    try:
        aux = scrapeo_autocompara(tipo, año, vehiculo, sexo, info_cliente)
        if len(aux) == 0:
            print(f'Fallo en la cotización 1 {vehiculo}, {cp}')
            Esperar_Cotizacion(5)
            aux = scrapeo_autocompara(tipo, año, vehiculo, sexo, info_cliente)
            print(f'Cotización exitosa 2 para {vehiculo}, {cp}')
            cotizaciones = pd.concat([cotizaciones,aux])
        else:
            print(f'Cotización exitosa 1 para {vehiculo}, {cp}')
            cotizaciones = pd.concat([cotizaciones,aux])
    except:
        print(f'Fallo en la cotización 2 {vehiculo}, {cp}')
        Esperar_Cotizacion(5)
        try:
            aux = scrapeo_autocompara(tipo, año, vehiculo, sexo, info_cliente)
            if len(aux) == 0:
                Esperar_Cotizacion(5)
                aux = scrapeo_autocompara(tipo, año, vehiculo, sexo, info_cliente)
                print(f'Cotización exitosa 4 para {vehiculo}, {cp}')
                cotizaciones = pd.concat([cotizaciones,aux])
            else:
                print(f'Cotización exitosa 3 para {vehiculo}, {cp}')
                cotizaciones = pd.concat([cotizaciones,aux])
        except:
            print(f'Fallo en la cotización 3 {vehiculo}, {cp}')
            pass
    cotizaciones.to_csv(os.path.join(ruta_carpeta,'Cotizados.csv'))

Cotización exitosa 1 para SENTRA SENSE AUT 1.8L 4CIL, 04000
Cotización exitosa 1 para VENTO STARLINE STD 1.6L 4CIL 4PTAS, 04000
Cotización exitosa 1 para MARCH ACTIVE STD 1.6L 4CIL, 04000
Cotización exitosa 1 para JETTA MK VI STD 2.0L 4CIL, 04000
Cotización exitosa 1 para JETTA CLASICO CL TIPTRONIC AUT 2.0L 4CIL A/A, 04000
Cotización exitosa 1 para CR-V EX AUT 2.4L 4CIL 5PTAS, 04000
Cotización exitosa 1 para MAZDA 3 SEDAN I STD 2.0L 4CIL 4PTAS, 04000
Cotización exitosa 1 para X-TRAIL SENSE 2 ROW CVT 2.5L 4CIL, 04000
Cotización exitosa 1 para RIO L STD 1.6L 4CIL 5PTAS, 04000
Fallo en la cotización 2 SPARK CLASSIC LT  B STD 1.2L 4CIL, 04000
...Intentando de nuevo en 5 min...
...Intentando de nuevo en 4 min...
...Intentando de nuevo en 3 min...
...Intentando de nuevo en 2 min...
...Intentando de nuevo en 1 min...
...Intentando de nuevo...
Cotización exitosa 3 para SPARK CLASSIC LT  B STD 1.2L 4CIL, 04000


In [11]:
cotizaciones

Aseguradora,DESCRIPCION,MODELO,TIPO,CP,SEXO,BIRTHDATE,Mas_Barato,Mejor_Servicio,ABA/CHUBB,AIG,ANA,ATLAS,AXA,GNP,HDI,INBURSA,MAPFRE,QUALITAS,ZURICH
0,SENTRA SENSE AUT 1.8L 4CIL,2017,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,9130.82,11983.76,11791.07,8943.62,11096.45,9994.77,10892.72,15175.2412,9163.98,9964.11,11810.11
0,VENTO STARLINE STD 1.6L 4CIL 4PTAS,2018,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,9046.19,10159.95,NaN,8735.35,12589.03,9817.58,8262.61,12533.6412,8108.79,12034.85,11558.81
0,MARCH ACTIVE STD 1.6L 4CIL,2018,AUTO,04000,M,20/05/1978,GNP,ABA/CHUBB,7883.71,9041.82,9872.72,8673.10,12523.78,7719.82,9769.69,10763.4120,8605.80,9379.86,10141.76
0,JETTA MK VI STD 2.0L 4CIL,2016,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,8354.25,11763.40,10020.88,8829.11,9736.10,9269.57,11145.09,14494.6412,7869.92,9997.91,9902.91
0,JETTA CLASICO CL TIPTRONIC AUT 2.0L 4CIL A/A,2013,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,6957.86,9450.19,9038.31,8081.74,10208.58,8833.44,9020.14,10866.8912,6319.36,8256.40,8594.88
0,CR-V EX AUT 2.4L 4CIL 5PTAS,2018,AUTO,04000,M,20/05/1978,AIG,ABA/CHUBB,13193.60,11459.26,13634.44,NaN,18341.55,14561.69,13241.49,22529.8412,11765.91,12686.86,17619.23
0,MAZDA 3 SEDAN I STD 2.0L 4CIL 4PTAS,2016,AUTO,04000,M,20/05/1978,GNP,ABA/CHUBB,12625.55,15628.11,NaN,12116.21,14303.70,8918.45,22339.32,15794.9812,10949.70,10851.64,15414.10
0,X-TRAIL SENSE 2 ROW CVT 2.5L 4CIL,2016,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,9354.61,13128.08,12608.97,9454.38,10767.63,9343.66,16642.17,21179.4412,8732.76,11804.92,10804.30
0,RIO L STD 1.6L 4CIL 5PTAS,2018,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,10277.85,10387.19,12822.92,10432.73,11561.04,9904.18,9954.73,13448.8812,9701.50,13214.70,13986.16
0,SPARK CLASSIC LT B STD 1.2L 4CIL,2017,AUTO,04000,M,20/05/1978,MAPFRE,ABA/CHUBB,8416.05,9140.94,NaN,7211.38,14771.94,7851.03,9949.34,10579.7212,7337.78,10274.19,10252.54


- 20% de las combinaciones con más peso
- De las 5000 ocupamos 1500 para las curvas, edad, etc. 
- Cotizarlo a inicios de mes la complete (5k) (Salir Con Fechas)
- Una/Dos veces a la semana 20 cotizaciones para la calendarización de los ajustes de tarifa de otras compañias